In [3]:
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'xlrd'

In [2]:
lockins = pd.read_excel('C:/Users/limzi/OneDrive/Forecasting & Reporting/IBP/inventory_data_for_python.xlsx', sheet_name='Unit Demand Actual').rename(columns = {'Sku': 'SKU'})
lockins_f = lockins.melt(id_vars=['SKU', 'Product Name'], var_name='Date', value_name='Quantity').drop('Product Name', axis =1)
print('lockins_f \n', lockins_f.head())


ImportError: Install xlrd >= 0.9.0 for Excel support

In [ ]:

sales_data = pd.read_excel('C:/Users/limzi/OneDrive/Forecasting & Reporting/IBP/inventory_data_for_python.xlsx', sheet_name='SalesQtyETS')
sales_data_f = sales_data.melt(id_vars=['SKU', 'Description'], var_name='Date', value_name='Quantity').drop('Description', axis =1).dropna()
print('sales_data_f \n', sales_data_f.head())

end_of_month = pd.read_excel('C:/Users/limzi/OneDrive/Forecasting & Reporting/IBP/inventory_data_for_python.xlsx', sheet_name='EOM1').rename(columns = {'SKU #': 'SKU'})
end_of_month_f = end_of_month.melt(id_vars=['SKU', 'Product Name'], var_name='Date', value_name='Quantity').drop('Product Name', axis =1)
print('end_of_month_f \n', end_of_month_f.head())

merge_table = end_of_month_f.merge(sales_data_f, how = 'left', on = ['SKU', 'Date'], suffixes=['EOM', 'Sales'])
merge_table = merge_table.merge(lockins_f, how = 'left', on = ['SKU', 'Date']).rename(columns = {'Quantity': 'LockIns'})
merge_table = merge_table.astype({'LockIns':'float64'})
merge_table = merge_table.sort_values(['SKU', 'Date'])
merge_table[(merge_table['SKU'] == "7770") | (merge_table['SKU'] == "2010") | (merge_table['SKU'] == "7870")]

In [91]:
merge_table[(merge_table['SKU'] == 7770) | (merge_table['SKU'] == 2010)].to_excel('merge_table_test.xlsx')
# test.to_excel('merge_table_test.xlsx')

In [49]:
merge_table.loc[0,'projected_eom'] = merge_table.loc[0, 'QuantityEOM'] - merge_table.loc[0, 'QuantitySales'] + merge_table.loc[0, 'LockIns']

In [107]:
merge_groupby = merge_table.groupby(['SKU', 'Date']).sum()
merge_groupby.apply(lambda x: x.QuantityEOM[0] - x.QuantitySales.cumsum() + x.LockIns)

AttributeError: ("'Series' object has no attribute 'QuantityEOM'", 'occurred at index QuantityEOM')

In [110]:
# merge_groupby.loc[1000, 'QuantityEOM']
merge_groupby_2 = merge_table.groupby(['SKU'])['QuantityEOM', 'QuantitySales', 'LockIns'].sum()
merge_groupby_2

,QuantityEOM,QuantitySales,LockIns
SKU,,,
1000,672.0,329.0,180.0
1001,596.0,126.0,180.0
1009,2024.0,268.0,250.0
1050,1130.0,372.0,252.0
1051,652.0,131.0,180.0
1057,0.0,0.0,0.0
1059,2401.0,458.0,250.0
1100,1364.0,161.0,288.0
1101,1616.0,256.0,288.0


In [67]:
merge_groupby.loc[0,'projected_eom'] = merge_groupby.loc[0, 'QuantityEOM'] - merge_groupby.loc[0, 'QuantitySales'] + merge_groupby.loc[0, 'LockIns']
merge_groupby

KeyError: 'the label [0] is not in the [index]'

In [ ]:
def rolling_apply(group):
    